<a href="https://colab.research.google.com/github/dernameistegal/airbnb_price/blob/main/data_utils/data_preparation/SavingDataInDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This simple introduction to the Airbnb data set(s) will give you a short overview over the available data. The city used for this introduction is Berlin, hence if you want to run the exact same notebook for a different city you would need to change a few minor details. Otherwise, if you have downloaded all necessary data sets and run this notebook in the same directory it should run smoothly.

### Index
1. Load data set
2. Price analysis
    * (Inspect reviews)
3. Main file (listings.csv.gz)
4. "Analyze" Images
5. "Analyze" Reviews
6. Calendar file
7.  neighbourhoods Geo.json file

In [1]:
#@title imports
%%capture
!pip install transformers
!pip install geopandas
import json
import os
import math
import pandas as pd
import gzip
from PIL import Image
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
import requests
from io import BytesIO
import matplotlib.image as mpimg
from tqdm import tqdm
from PIL import Image
import numpy as np

from shapely.geometry import Point, Polygon

import seaborn as sns

from transformers import pipeline

import folium
from folium.plugins import FastMarkerCluster
from branca.colormap import LinearColormap

In [2]:
#@title mount drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# make directories in drive
!mkdir -p /content/drive/MyDrive/data/data1/
!mkdir -p  /content/drive/MyDrive/data/hostpics/
!mkdir -p  /content/drive/MyDrive/data/thumbnails/

In [3]:
# load data to drive
!wget http://data.insideairbnb.com/austria/vienna/vienna/2021-11-07/data/listings.csv.gz
!wget http://data.insideairbnb.com/austria/vienna/vienna/2021-11-07/data/calendar.csv.gz
!wget http://data.insideairbnb.com/austria/vienna/vienna/2021-11-07/data/reviews.csv.gz
!wget http://data.insideairbnb.com/austria/vienna/vienna/2021-11-07/visualisations/listings.csv
!wget http://data.insideairbnb.com/austria/vienna/vienna/2021-11-07/visualisations/reviews.csv
!wget http://data.insideairbnb.com/austria/vienna/vienna/2021-11-07/visualisations/neighbourhoods.csv
!wget http://data.insideairbnb.com/austria/vienna/vienna/2021-11-07/visualisations/neighbourhoods.geojson

--2022-02-03 00:04:34--  http://data.insideairbnb.com/austria/vienna/vienna/2021-11-07/data/listings.csv.gz
Resolving data.insideairbnb.com (data.insideairbnb.com)... 52.216.65.138
Connecting to data.insideairbnb.com (data.insideairbnb.com)|52.216.65.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744167 (6.4M) [application/x-gzip]
Saving to: ‘listings.csv.gz.1’

listings.csv.gz.1   100%[===================>]   6.43M  26.9MB/s    in 0.2s    

2022-02-03 00:04:35 (26.9 MB/s) - ‘listings.csv.gz.1’ saved [6744167/6744167]

--2022-02-03 00:04:35--  http://data.insideairbnb.com/austria/vienna/vienna/2021-11-07/data/calendar.csv.gz
Resolving data.insideairbnb.com (data.insideairbnb.com)... 52.216.65.138
Connecting to data.insideairbnb.com (data.insideairbnb.com)|52.216.65.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11336790 (11M) [application/x-gzip]
Saving to: ‘calendar.csv.gz.1’

calendar.csv.gz.1   100%[===================>]  

In [158]:
#read files and show header for overview
listings = pd.read_csv("listings.csv")
reviews = pd.read_csv("reviews.csv")
listings_meta = pd.read_csv("listings.csv.gz")
reviews_meta = pd.read_csv("reviews.csv.gz")
calendar = pd.read_csv("calendar.csv.gz")
listings_meta.set_index("id", inplace=True)

# Save Images (only run once)

In [43]:

# descriptive statistics for availability of pictures
n_no_hostpic = sum(listings_meta["host_picture_url"].isnull()) 
n_no_thumbnail = sum(listings_meta["picture_url"].isnull())
n_hosts_no_thumbnail = len(np.unique(listings_meta["host_id"][listings_meta["host_picture_url"].isnull()]))
print(f"{n_no_hostpic} listings have no hostpic. In total, {n_hosts_no_thumbnail} hosts have no hostpic. {n_no_thumbnail} listings have no thumbnail.")

22 listings have no hostpic. In total, 6 hosts have no hostpic. 0 listings have no thumbnail.


In [74]:
# instantiate list of all ids where url does not work
pic_malfunction = []

# save hostpics that are available
for i in range(1488, 1505):

    # get url
    test = listings_meta.loc[i]["host_picture_url"]
    break

    # check if url is not available
    if pd.isna(url):
        continue

    # scrape url
    response = requests.get(url)

    # check if url does not work
    try:
        img_plot = Image.open(BytesIO(response.content)).resize(IMAGE_SIZE)
    except:
        pic_malfunction.append(listings_meta.loc[i]["id"])
        continue
    
    # save rgb data
    rgb_data = np.array(img_plot)
    save_path = "/content/drive/MyDrive/data/hostpics/hostpic" + str(listings_meta.loc[i]["id"])
    np.save(save_path, rgb_data)

In [ ]:
# save ids where host pics are not available (either no url or non-functioning url) in dictionary
nopic = np.unique(listings_meta["id"][listings_meta["host_picture_url"].isnull()])
nopic = list(nopic)
#indices = nopic + pic_malfunction

missing_data = {"hostpic": [int(ind) for ind in indices]}

temp_file = open("/content/drive/MyDrive/data/missing_data.json", "w")
json.dump(missing_data, temp_file)
temp_file.close()

#temp_file = open("missing_data.json", "r")
#output = json.load(temp_file)

In [ ]:
# instantiate list of all ids where url does not work
pic_malfunction = []

# save thumbnails that are available
for i in range(len(listings_meta)):

    # get url
    url = listings_meta.loc[i]["picture_url"]

    # check if url is not available
    if pd.isna(url):
        continue

    # scrape url
    response = requests.get(url)

    # check if url does not work
    try:
        img_plot = Image.open(BytesIO(response.content)).resize(IMAGE_SIZE)
    except:
        pic_malfunction.append(listings_meta.loc[i]["id"])
        continue
    
    # save rgb data
    rgb_data = np.array(img_plot)
    save_path = "/content/drive/MyDrive/data/thumbnails/thumbnail" + str(listings_meta.loc[i]["id"])
    np.save(save_path, rgb_data)

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8589934590 bytes but only got 16480. Skipping tag 34855
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (103168284 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [ ]:
# save ids where thumbnails are not available in dictionary 
# various reasons, e.g. could not load because of corrupt exif data or image size

temp_file = open("/content/drive/MyDrive/data/missing_data.json", "r")
temp_file_dict = json.load(temp_file)
temp_file_dict["thumbnail"] = pic_malfunction
temp_file.close()

temp_file = open("/content/drive/MyDrive/data/missing_data.json", "w")
json.dump(temp_file_dict, temp_file)
temp_file.close()

#temp_file = open("missing_data.json", "r")
#output = json.load(temp_file)

# try to get urls (not necessary anymore)

In [ ]:
# does not have to be run again
urls = []

for index in tqdm(listings_meta.index):

    # get url_temp
    index = pd.Index([index])
    url_temp = listings_meta["host_picture_url"].loc[index].values[0]

    if not pd.isna(url_temp):
        url = requests.get(url_temp)
        print(url)
        urls.append(pd.Series([url], index=index))

    else:
        urls.append(pd.Series([np.nan], index=index))

temp_urls_series = pd.concat(urls, axis=0)
temp_urls_series.name = "temp_hostpics_url"
path = ("/content/drive/MyDrive/Colab/airbnb/data/hostpics/hostpics_url.pickle")
temp_urls_series.to_pickle(path)

In [127]:
temp_urls_series = pd.read_pickle("/content/drive/MyDrive/Colab/airbnb/data/hostpics/hostpics_url.pickle")

In [134]:
# does not have to be run again
links = []

for index in tqdm(temp_urls_series.index):
     
    index = pd.Index([index])
    obj = temp_urls_series.loc[index].values[0]

    if not pd.isna(obj):
        link = obj.url
        links.append(pd.Series([link], index=index))
    else:
        links.append(pd.Series([np.nan], index=index))

urls_series = pd.concat(links, axis=0)
urls_series.name = "hostpics_url"
urls_series = pd.concat([temp_urls_series, urls_series], axis=1)
path = ("/content/drive/MyDrive/Colab/airbnb/data/hostpics/hostpics_url.pickle")
urls_series.to_pickle(path)

100%|██████████| 11409/11409 [00:06<00:00, 1676.19it/s]


In [ ]:
# https://a0.muscache.com/im/users/14368032/profile_pic/1398641174/original.jpg?
# https://a0.muscache.com/im/users/6196667/profile_pic/1387179693/original.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/61c74b99-2e58-4bc4-9e50-45f8b7d6e269.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/04a419de-c13a-4511-9750-6133312e101a.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/07077e9d-adb7-46c2-8d6d-c29e184f9a2b.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240
# https://a0.muscache.com/im/pictures/user/8e2de99c-a731-41f8-89eb-0a3d31608ae9.jpg?im_w=240

In [159]:
listings_meta.loc[urls_series.hostpics_url.isna()]

,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2808957,https://www.airbnb.com/rooms/2808957,20211107161644,2021-11-08,Cozy flat in good area,- Good connection to public transport<br />- R...,NaN,https://a0.muscache.com/pictures/1b4880c7-f10a...,14368032,https://www.airbnb.com/users/show/14368032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,Margareten,NaN,48.189280,16.349970,Entire rental unit,Entire home/apt,4,NaN,1 bath,1.0,2.0,"[""Essentials"", ""Washer"", ""Wifi"", ""Elevator"", ""...",$74.00,2,7,2,2,7,7,2.0,7.0,NaN,t,0,3,3,270,2021-11-08,45,5,1,2014-06-03,2017-01-02,4.81,4.76,4.83,4.95,4.95,4.80,4.78,NaN,f,1,1,0,0,0.50
7079941,https://www.airbnb.com/rooms/7079941,20211107161644,2021-11-07,Stylish & Central Vienna Apartment,"Wonderful historic apartment of an artist, bui...",The sixth district in Vienna belongs to the mo...,https://a0.muscache.com/pictures/89801816/8814...,6196667,https://www.airbnb.com/users/show/6196667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,"Vienna, Wien, Austria",Mariahilf,NaN,48.193360,16.345960,Entire rental unit,Entire home/apt,3,NaN,1 bath,1.0,2.0,"[""Essentials"", ""Smoke alarm"", ""Washer"", ""Wifi""...",$62.00,1,1125,1,1,1125,1125,1.0,1125.0,NaN,t,0,0,0,0,2021-11-07,1,0,0,2015-09-05,2015-09-05,5.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,f,1,1,0,0,0.01
8808555,https://www.airbnb.com/rooms/8808555,20211107161644,2021-11-08,Dachwohnung mit Wintergarten,In dieser hellen eingerichteten Dachwohnung li...,NaN,https://a0.muscache.com/pictures/9eb4c444-09d7...,46149278,https://www.airbnb.com/users/show/46149278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,Leopoldstadt,NaN,48.224470,16.386960,Entire rental unit,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Wifi"", ""Washer"", ""Elevator"", ""Dedicated work...",$80.00,10,90,10,10,90,90,10.0,90.0,NaN,t,30,60,90,365,2021-11-08,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
13880640,https://www.airbnb.com/rooms/13880640,20211107161644,2021-11-07,"Nice Room, near public transportation",Meine Unterkunft ist in der Nähe von Heurigen ...,NaN,https://a0.muscache.com/pictures/0032f728-5707...,43595573,https://www.airbnb.com/users/show/43595573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,Dbling,NaN,48.246770,16.361040,Private room in rental unit,Private room,1,NaN,1 bath,1.0,1.0,"[""Kitchen"", ""Wifi"", ""Elevator"", ""Long term sta...",$14.00,2,30,2,2,30,30,2.0,30.0,NaN,t,0,0,0,0,2021-11-07,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
21632198,https://www.airbnb.com/rooms/21632198,20211107161644,2021-11-07,Walk to Schönbrunn Palace 

# save raw hostpics as jpg

In [ ]:
os.listdir("/content/drive/MyDrive/Colab/airbnb/data/hostpics/hostpics_raw")

In [5]:
exceptions = []

for pic_name in tqdm(os.listdir("/content/drive/MyDrive/Colab/airbnb/data/hostpics/hostpics_raw")):
    pic_path = "/content/drive/MyDrive/Colab/airbnb/data/hostpics/hostpics_raw/" + pic_name
    pic = np.load(pic_path)
    pic_name = pic_name[7:-4]

    try:
        pic = Image.fromarray(pic)
        pic.save("/content/drive/MyDrive/Colab/airbnb/data/hostpics/hostpics_raw_jpeg/" + pic_name + ".jpg")
    except:
        exceptions.append("hostpic" + pic_name + ".npy") 


100%|██████████| 11375/11375 [02:47<00:00, 68.11it/s]


In [30]:
test = np.load("/content/drive/MyDrive/Colab/airbnb/data/hostpics/hostpics_raw/" + exceptions[1])

In [24]:
test2 = np.load("/content/drive/MyDrive/Colab/airbnb/data/hostpics/hostpics_raw/" + os.listdir("/content/drive/MyDrive/Colab/airbnb/data/hostpics/hostpics_raw")[0])

In [19]:
test2.dtype

dtype('uint8')

In [31]:
test = test.astype("uint8")
test.shape

(224, 224, 3)

In [32]:
test = Image.fromarray(test)